In [22]:
from currency_symbols import CurrencySymbols
import string
import re
import os
from tqdm import tqdm
import pandas as pd
from google.cloud import storage
from google.cloud import bigquery
from google.oauth2 import service_account

# Define the regex patterns
emoji_pattern = (
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
)
accented_characters = u"\u00C0-\u00FF"  # Latin-1 Supplement (accented characters)
special_symbols = '©®™±≠≤≥∞π∑§¶†•′″‰′′←→↑↓↔↕'

# ISO codes for all currencies 
currencies = [
    'AFN', 'EUR', 'ALL', 'DZD', 'USD', 'EUR', 'AOA', 'XCD', 'XCD', 'ARS', 'AMD', 'AWG', 'AUD', 'EUR', 'AZN',
    'BSD', 'BHD', 'BDT', 'BBD', 'BYN', 'EUR', 'BZD', 'XOF', 'BMD', 'INR', 'BTN', 'BOB', 'BOV', 'USD', 'BAM',
    'BWP', 'NOK', 'BRL', 'USD', 'BND', 'BGN', 'XOF', 'BIF', 'CVE', 'KHR', 'XAF', 'CAD', 'KYD', 'XAF', 'XAF',
    'CLP', 'CLF', 'CNY', 'AUD', 'AUD', 'COP', 'COU', 'KMF', 'CDF', 'XAF', 'NZD', 'CRC', 'XOF', 'EUR', 'CUP',
    'CUC', 'ANG', 'EUR', 'CZK', 'DKK', 'DJF', 'XCD', 'DOP', 'USD', 'EGP', 'SVC', 'USD', 'XAF', 'ERN', 'EUR',
    'SZL', 'ETB', 'EUR', 'FKP', 'DKK', 'FJD', 'EUR', 'EUR', 'EUR', 'XPF', 'EUR', 'XAF', 'GMD', 'GEL', 'EUR',
    'GHS', 'GIP', 'EUR', 'DKK', 'XCD', 'EUR', 'USD', 'GTQ', 'GBP', 'GNF', 'XOF', 'GYD', 'HTG', 'USD', 'AUD',
    'EUR', 'HNL', 'HKD', 'HUF', 'ISK', 'INR', 'IDR', 'XDR', 'IRR', 'IQD', 'EUR', 'GBP', 'ILS', 'EUR', 'JMD',
    'JPY', 'GBP', 'JOD', 'KZT', 'KES', 'AUD', 'KPW', 'KRW', 'KWD', 'KGS', 'LAK', 'EUR', 'LBP', 'LSL', 'ZAR',
    'LRD', 'LYD', 'CHF', 'EUR', 'EUR', 'MOP', 'MKD', 'MGA', 'MWK', 'MYR', 'MVR', 'XOF', 'EUR', 'USD', 'EUR',
    'MRU', 'MUR', 'EUR', 'XUA', 'MXN', 'MXV', 'USD', 'MDL', 'EUR', 'MNT', 'EUR', 'XCD', 'MAD', 'MZN', 'MMK',
    'NAD', 'ZAR', 'AUD', 'NPR', 'EUR', 'XPF', 'NZD', 'NIO', 'XOF', 'NGN', 'NZD', 'AUD', 'USD', 'NOK', 'OMR',
    'PKR', 'USD', 'PAB', 'USD', 'PGK', 'PYG', 'PEN', 'PHP', 'NZD', 'PLN', 'EUR', 'USD', 'QAR', 'EUR', 'RON',
    'RUB', 'RWF', 'EUR', 'SHP', 'XCD', 'XCD', 'EUR', 'EUR', 'XCD', 'WST', 'EUR', 'STN', 'SAR', 'XOF', 'RSD',
    'SCR', 'SLE', 'SGD', 'ANG', 'XSU', 'EUR', 'EUR', 'SBD', 'SOS', 'ZAR', 'SSP', 'EUR', 'LKR', 'SDG', 'SRD',
    'NOK', 'SEK', 'CHF', 'CHE', 'CHW', 'SYP', 'TWD', 'TJS', 'TZS', 'THB', 'USD', 'XOF', 'NZD', 'TOP', 'TTD',
    'TND', 'TRY', 'TMT', 'USD', 'AUD', 'UGX', 'UAH', 'AED', 'GBP', 'USD', 'USD', 'USN', 'UYU', 'UYI', 'UYW',
    'UZS', 'VUV', 'VES', 'VED', 'VND', 'USD', 'USD', 'XPF', 'MAD', 'YER', 'ZMW', 'ZWL', 'ZWG', 'XBA', 'XBB',
    'XBC', 'XBD', 'XTS', 'XXX', 'XAU', 'XPD', 'XPT', 'XAG'
]

currency_symbols = [CurrencySymbols.get_symbol(x) for x in currencies]
currency_symbols = [x for x in currency_symbols if x and not x.isalpha()]  # Filter out None values and alphabetic characters

pattern_string = (
    r'[^a-zA-Z0-9\s' +
    ''.join([re.escape(x) for x in string.punctuation]) +
    ''.join([re.escape(x) for x in currency_symbols]) +
    special_symbols +
    emoji_pattern +
    accented_characters +
    ']'
)

pattern = re.compile(pattern_string, re.UNICODE)

def clean_dataframe(df):
    #df_cleaned = df[~df.apply(lambda row: row.astype(str).str.contains(pattern).any(), axis=1)]
    #df_cleaned['text'] = df_cleaned['text'].str.replace('"', ' ')
    #df_cleaned['text'] = df_cleaned['text'].str.strip()
    #df_cleaned['text'] = df_cleaned['text'].str.rstrip()
    #escaped_chars_pattern = re.compile(r'[' + ''.join([re.escape(c) for c in ['\\', '\t', '\n']]) + ']')
    #df_cleaned['text'] = df_cleaned['text'].str.replace(escaped_chars_pattern, ' ', regex=True)
    #excessive_whitespace_pattern = re.compile(r'\s{2,}')
    #df_cleaned['text'] = df_cleaned['text'].str.replace(excessive_whitespace_pattern, ' ', regex=True)
    #df_cleaned.drop([], axis=1, inplace=True)
    df_cleanded = df
    return df_cleaned


def process_and_clean_csv(input_path):
    df = pd.read_csv(input_path, header=0, index_col=0, encoding='ISO-8859-1', on_bad_lines='skip')
    df_cleaned = clean_dataframe(df)
    return df_cleaned

### We ultimately decided not to go with uploading a csv to a bucket of our projects. We will instead 
# feed the df into a bigquery table directly
#def upload_file(bucket_name, source_file_path, destination_blob_name):
#    bucket = storage_client.bucket(bucket_name)
#    blob = bucket.blob(destination_blob_name)
#    storage.blob._DEFAULT_CHUNKSIZE = 100 * 1024 * 1024  # 100 MB chunk size
#    storage.blob._MAX_MULTIPART_SIZE = 100 * 1024 * 1024  # 100 MB max multipart size
#    blob.upload_from_filename(source_file_path, timeout=600)
#    print(f'{source_file_path} uploaded to {bucket_name} as {destination_blob_name}.')

def load_to_bigquery(df, table_ref):
    job = bigquery_client.load_table_from_dataframe(df, table_ref)
    job.result()
    print(f'Loaded DataFrame into {table_ref.table_id}.')

# Service account key
key_path = '/Users/chkapsalis/Downloads/nlp-project-427710-3e1a48df3dba.json'
credentials = service_account.Credentials.from_service_account_file(key_path)

# Google Cloud project id and dataset information
project_id = 'nlp-project-427710'
dataset_id = 'person_emotion' 
table_id = 'MBTI_personality' 

# Initialization of the BigQuery client
bigquery_client = bigquery.Client(project=project_id, credentials=credentials)
storage_client = storage.Client(project=project_id, credentials=credentials)

# Create the dataset if it does not exist
dataset_ref = bigquery_client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)

try:
    bigquery_client.get_dataset(dataset_ref)  # Make an API request.
    print(f"Dataset {dataset_id} already exists.")
except:
    dataset = bigquery_client.create_dataset(dataset)  # Make an API request.
    print(f"Dataset {dataset_id} created.")

# Define the job configuration for loading data
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("text", "STRING"),
        bigquery.SchemaField("label", "STRING")
    ],
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    field_delimiter='|',  # used this custom delimiter to help make sense out of the data
    autodetect=False,  # Automatically detect the schema
    max_bad_records=2000,  # Allow up to 2000 bad records
    ignore_unknown_values=True  # Ignore unknown values
)

source_folder = '/Users/chkapsalis/Downloads/person_emotion'
cleaned_folder = '/Users/chkapsalis/Downloads/cleaned_person_emotion'

os.makedirs(cleaned_folder, exist_ok=True)

for filename in tqdm(os.listdir(source_folder)):
    if filename.endswith('.csv'):
        source_file = os.path.join(source_folder, filename)
        df_cleaned = pd.read_csv(source_file, header=0, index_col=0, encoding='ISO-8859-1', on_bad_lines='skip')

        # making sure that the dtypes of columns match what asserted in the schema
        #df_cleaned[''] = pd.to_numeric(df_cleaned['user_followers'], errors='coerce')
        table_id = {
            'twitter_MBTI.csv': 'MBTI_personality',
            #'text.csv': 'emotions1'
        }
        
        # Upload the cleaned DataFrame to BigQuery
        #load_to_bigquery(df_cleaned, dataset_ref.table(table_id))
        load_to_bigquery(df_cleaned, dataset_ref.table(table_id[filename]))


Dataset person_emotion already exists.


100%|███████████████████████████████████████████████████████████████████████| 2/2 [01:41<00:00, 50.50s/it]

Loaded DataFrame into MBTI_personality.


In [20]:
df_cleaned

,text,label
0,@Pericles216 @HierBeforeTheAC @Sachinettiyil T...,intj
1,@Hispanthicckk Being you makes you look cute||...,intj
2,@Alshymi Les balles sont rÃ©elles et sont tirÃ...,intj
3,"I'm like entp but idiotic|||Hey boy, do you wa...",intj
4,@kaeshurr1 Give it to @ZargarShanif ... He has...,intj
...,...,...
7806,"@sobsjjun God,,pls take care ð|||@sobsjjun ...",intp
7807,@Ignis_02 wow last time i got intp https://t.c...,intp
7808,@akupilled A 100%|||@akupilled That SOMEONE wi...,entp
7809,If youâre #INTJ this one is for you | What i...,infj


In [21]:
df_cleaned.shape

(7811, 2)